### Import cw-simulate module and create new instance

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";

senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
client = new SimulateCosmWasmClient({
  chainId: "Oraichain",
  bech32Prefix: "orai"
});


### Setup account balance

In [ ]:
import { coin } from "@cosmjs/stargate";

client.app.bank.setBalance(senderAddress, [coin('1000000000', 'orai'), coin('1000000000', 'usdt')]);

console.log(client.app.bank.getBalance(senderAddress));

### Deploy USDT token contract

In [ ]:
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/wasm/cw20-base.wasm").then(res=>res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(senderAddress, codeId, {
    decimals: 6,
    symbol: "USDT",
    name: "USDT token",
    mint: { minter: senderAddress },
    initial_balances: [{ address: senderAddress, amount: "1000000000" }] 
}, "cw20 token", "auto");
usdtContract = new Cw20BaseClient(client, senderAddress, contractAddress);

console.log(await usdtContract.tokenInfo());

### Deploy Orderbook contract

In [ ]:
import { OraiswapLimitOrderClient } from "@oraichain/oraidex-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/wasm/orderbook.wasm").then(res=>res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(senderAddress, codeId, {
    admin: senderAddress,
    version: '0.0.1',
    name: 'Orderbook'
}, "limit order", "auto");
orderbookContract = new OraiswapLimitOrderClient(client, senderAddress, contractAddress);

console.log(await orderbookContract.contractInfo());

### Create native orderbook pair

In [ ]:
import { coins } from "@cosmjs/stargate";

const result = await orderbookContract.createOrderBookPair({
    baseCoinInfo: { native_token: { denom: "orai" } },
    quoteCoinInfo: { native_token: { denom: "usdt" } },
    spread: '0.5',
    minQuoteCoinAmount: '10'
}, 'auto', '', coins('20', 'orai'));

console.log(result);

const queryPair = await orderbookContract.orderBook({
    assetInfos: [
        { native_token: { denom: "orai" } }, 
        { native_token: { denom: "usdt" } }
    ]
});

console.log("orai-usdt pair:", queryPair);

### Create cw20 orderbook pair

In [ ]:
import { coins } from "@cosmjs/stargate";

await orderbookContract.createOrderBookPair({
      baseCoinInfo: { native_token: { denom: "orai" } },
      quoteCoinInfo: { token: { contract_addr: usdtContract.contractAddress } },
      spread: '0.1',
      minQuoteCoinAmount: '1'
}, 'auto', '', coins('20', 'orai'));

const queryPair = await orderbookContract.orderBook({
    assetInfos: [
        { native_token: { denom: "orai" } }, 
        { token: { contract_addr: usdtContract.contractAddress } }
    ]
});

console.log("orai-cw20-usdt pair:", queryPair);

### Submit native token order

In [ ]:
import { coin } from "@cosmjs/stargate";

const orders = [
    [coin('567', 'orai'), coin('123', 'usdt'), 'buy'],
    [coin('654', 'orai'), coin('111', 'usdt'), 'buy'],
    [coin('553', 'orai'), coin('100', 'usdt'), 'sell'],
    [coin('632', 'orai'), coin('100', 'usdt'), 'sell']
];

for (const [base, quote, direction] of orders) {
    await orderbookContract.submitOrder(
      {
        assets: [
            { info: { native_token: { denom: base.denom } }, amount: base.amount }, 
            { info: { native_token: { denom: quote.denom } }, amount: quote.amount }
        ],
        direction
      },
      'auto',
      '',
      [direction === 'sell' ? base : quote]     
    );
}

// query an order
const queryOrder = await orderbookContract.order({
  assetInfos: [
    { native_token: { denom: "orai" } }, 
    { native_token: { denom: "usdt" } }, 
  ],
  orderId: 1
});

console.log(queryOrder);

### Submit cw20 token order

In [ ]:
import { toBinary } from "@cosmjs/cosmwasm-stargate";

await usdtContract.send({
    amount: '7000000',
    contract: orderbookContract.contractAddress,
    msg: toBinary({
      submit_order: {
        direction: 'buy',
        assets: [
            { info: { native_token: { denom: "orai" } }, amount: "2000000" },  
            { info: { token: { contract_addr: usdtContract.contractAddress } }, amount: "7000000" }
        ]
      }
    })
});

const balanceRes = await usdtContract.balance({ address: senderAddress });
// expect balance to be 993000000

console.log(balanceRes);

### Cancel native order

In [ ]:
await orderbookContract.cancelOrder({
    assetInfos: [
        { native_token: { denom: "orai" } }, 
        { native_token: { denom: "usdt" } }, 
    ],
    orderId: 1
});

try {
    await orderbookContract.order({
      assetInfos: [
        { native_token: { denom: "orai" } }, 
        { native_token: { denom: "usdt" } }, 
      ],
      orderId: 1
    });
} catch (ex) {
    // expect oraiswap_limit_order::orderbook::Order not found
    console.log(ex.message);
}

### Cancel cw20 order

In [ ]:
await orderbookContract.cancelOrder({
    assetInfos: [
      { native_token: { denom: 'orai' } },
      { token: { contract_addr: usdtContract.contractAddress } }
    ],
    orderId: 5
});

const balanceRes = await usdtContract.balance({ address: senderAddress });

// expect balance to be 1000000000
console.log(balanceRes);

### Matching orders

In [ ]:
let queryAll = await orderbookContract.orders({
    assetInfos: [
        { native_token: { denom: "orai" } }, 
        { native_token: { denom: "usdt" } }, 
    ],
    filter: {
      bidder: senderAddress
    }
});
// have 3 orders
console.log('pending orders before matching', queryAll.orders.map(order=>order.order_id));

await orderbookContract.executeOrderBookPair({
    assetInfos: [
        { native_token: { denom: "orai" } }, 
        { native_token: { denom: "usdt" } }, 
    ]
})

queryAll = await orderbookContract.orders({
    assetInfos: [
        { native_token: { denom: "orai" } }, 
        { native_token: { denom: "usdt" } }, 
    ],
    filter: {
      bidder: senderAddress
    }
});

// now have 2 orders
console.log('pending orders after matching', queryAll.orders.map(order=>order.order_id));